In [1]:
print("hi")

hi


In [3]:
import cv2
import numpy as np
from paint_init import *

class PaintApp:
    def __init__(self):
        self.mouse_mode = 0
        self.draw_mode = 0
        self.pt1 = (0,0)
        self.pt2 = (0,0)
        self.Color = (0,0,0)
        self.thickness = 3
        self.image = np.full((500,800,3), 255 ,np.uint8)

        self.icons = self.place_icons(self.image, (60,60))
        x,y,w,h = self.icons[-1]
        menu_width = w*2

        self.icons.append((0, y+h+2, 120, 120))
        self.icons.append((0, y+h+124, 120,15))

        self.create_colorPlatte(self.image, 0, self.icons[PALETTE])
        self.create_hueIndex(self.image,self.icons[HUE_IDX])

        self.canvas = self.image[:,menu_width:self.image.shape[1]]

        cv2.imshow("PaintCV", self.image)
        cv2.setMouseCallback("PaintCV", self.onMouse)
        cv2.createTrackbar("Thickness", "PaintCV", self.thickness, 255, lambda v:self.onTrackbar(v))

    
    def place_icons(self,image,size):
        icon_name = ["rect", "circle", "eclipe", "line", "brush", "eraser", "open","save","plus","minus","clear","color"]
        icons = [( i%2, i//2, 1, 1) for i in range(len(icon_name))]
        icons = np.multiply(icons, size[0])

        for roi, name in zip(icons, icon_name):
            icon = cv2.imread(f"icon/{name}.jpg", cv2.IMREAD_COLOR)
            if icon is None:
                print(f"Icon image file not found: {name}.jpg")
                continue
            x,y,w,h = roi
            image[y:y+h, x:x+w] = cv2.resize(icon, size)
        return list(icons)
    
    def create_colorPlatte(self, image, idx,roi):
        x,y,w,h = roi
        hue = idx-x
        palette = [[(hue,j,h-1-i) for j in range(w)] for i in range(h)]

        ratios = (180/w, 255/w, 255/h)
        palette = np.multiply(palette,ratios).astype('uint8')

        image[y:y+h,x:x+w] = cv2.cvtColor(palette, cv2.COLOR_HSV2BGR)

    def create_hueIndex(self, image, roi):
        x,y,w,h = roi
        index = [[(j,1,1) for j in range(w)] for i in range(h)]
        ratios = (180/w,255,255)
        hueIndex = np.multiply(index, ratios).astype('uint8')

        image[y:y+h,x:x+w] = cv2.cvtColor(hueIndex, cv2.COLOR_HSV2BGR)

    def onMouse(self, event, x, y, flags, param=None):
        if event == cv2.EVENT_LBUTTONUP:
            for i, (x0,y0,w,h) in enumerate(self.icons):
                if x0<=x <x0+w and y0 <=y<y0+h:
                    if i<6:
                        self.mouse_mode = 1
                        self.draw_mode = i
                    else:
                        self.command(i)
                    return
            self.pt2 = (x,y)
            self.mouse_mode = 1

        elif event == cv2.EVENT_LBUTTONDOWN:
            self.pt1 = (x,y)
            self.mouse_mode = 2
        
        if self.mouse_mode >=2:
            self.mouse_mode = 0 if x<125 else 3
            self.pt2 = (x,y)

    def onTrackbar(self, value):
        self.mouse_mode = 0
        self.thickness = value

    def draw(self,image,color=(200,200,200)):
        if self.draw_mode == DRAW_RECTANGLE:
            cv2.rectangle(image, self.pt1, self.pt2, color, self.thickness)
        elif self.draw_mode == DRAW_LINE:
            cv2.line(image, self.pt1, self.pt2, color, self.thickness)
        elif self.draw_mode == DRAW_BRUSH:
            cv2.line(image, self.pt1, self.pt2, color, self.thickness*3)
            self.pt1 = self.pt2
        elif self.draw_mode == ERASE:
            cv2.line(image, self.pt1, self.pt2, (255,255,255), self.thickness*5)
            self.pt1 = self.pt2
        elif self.draw_mode == DRAW_CIRCE:
            d = np.subtract(self.pt1, self.pt2)
            radius = int(np.sqrt(d[0]**2+d[1]**2))
            cv2.circle(image, self.pt2, radius, color, self.thickness)
        elif self.draw_mode == DRAW_ECLIPSE:
            center = np.abs(np.add(self.pt1, self.pt2))//2
            size = np.abs(np.subtract(self.pt1,self.pt2))//2
            cv2.ellipse(image,center,size,0,0,360,color,self.thickness)

        cv2.imshow("PaintCV", image)

    def command(self, mode):
        if mode == PALETTE:
            pixel = self.image[self.pt2[1], self.pt2[0]]
            x,y,w,h = self.icons[COLOR]
            self.image[y:y+h-1, x:x+w-1] = pixel
            self.Color = tuple(map(int,pixel))

        elif mode == HUE_IDX:
            self.create_colorPlatte(self.image, self.pt2[0], self.icons[PALETTE])
        
        elif mode == OPEN:
            tmp = cv2.imread("img/img.jpg", cv2.IMREAD_COLOR)
            if tmp is not None:
                resized = cv2.resize(tmp, self.canvas.shape[1::-1])
                self.canvas[:,:] = resized

        elif mode == SAVE:
            cv2.imwrite("img/paintcv_result.jpg", self.canvas)

        elif mode == PLUS:
            val = np.full(self.canvas.shape, 10, np.uint8)
            cv2.add(self.canvas, val, self.canvas)
        
        elif mode == MINUS:
            val = np.full(self.canvas.shape, 10, np.uint8)
            cv2.subtract(self.canvas, val, self.canvas)


        elif mode == CREAR:
            self.canvas[:] = (255,255,255)
            self.mouse_mode = 0

        cv2.imshow("PaintCV", self.canvas)

    def run(self):
        while True:
            if self.mouse_mode == 1:
                self.draw(self.image, self.Color)
            elif self.mouse_mode==3:
                if self.draw_mode in (DRAW_BRUSH,ERASE):
                    self.draw(self.image, self.Color)
                else:
                    self.draw(self.image.copy(), (200,200,200))
            
            if cv2.waitKey(30) == 27:
                break
        cv2.destroyAllWindows()


if __name__ == "__main__":
    app = PaintApp()
    app.run()

         

                  

In [8]:
import cv2
import numpy as np

def preprocessing(no):
    image = cv2.imread("face/%2d.jpg"%no, cv2.IMREAD_COLOR)
    if image is None : return None, None
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    return image, gray

face_cascade = cv2.CascadeClassifier(r"haarcascades/haarcascade_frontalface_alt2.xml")
eye_cascade = cv2.CascadeClassifier(r"haarcascades/haarcascade_eye.xml")
image, gray = preprocessing(34)
if image is None : raise Exception(f"에러")

faces = face_cascade.detectMultiScale(gray, 1.1, 2, 0, (100,100))

if faces.any():
    x,y,w,h = faces[0]
    face_image = image[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(face_image, 1.15, 7, 0, (25,20))
    if len(eyes)==2:
        for ex, ey, ew, eh in eyes:
            center = (x+ex+ew//2, y+ey+eh//2)
            cv2.circle(image,center, 10, (0,255,0),2)
    else:
        print("미검출")
    cv2.rectangle(image,faces[0],(255,0,0),2)
    cv2.imshow("image",image)

else : print("눈 미검출")

cv2.waitKey(0)


-1

In [9]:
face_cascade = cv2.CascadeClassifier(r"haarcascades/haarcascade_frontalface_alt2.xml")
image, gray = preprocessing(10)
if image is None : raise Exception("이미지 없음")

faces = face_cascade.detectMultiScale(gray, 1.1, 2, 0, (100,100))
if len(faces)>0:
    for (x,y,w,h) in faces:
        face_roi = image[y:y+h,x:x+w]

        scale = 0.07

        small = cv2.resize(face_roi, dsize=None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
        mosaic = cv2.resize(small, (w,h), interpolation=cv2.INTER_NEAREST)
        image[y:y+h, x:x+w] = mosaic

    cv2.imshow("mosaic face", image)
else:
    print("미검출")
    cv2.imshow("mosaic face", image)

cv2.waitKey(0)
cv2.destroyAllWindows()
